
wind_size = 60
stride_size = 5
b_size = 16
acuu- 71

<br>

wind_size = 60
stride_size = 10
b_size = 16
acuu- 70

<br>

wind_size = 120
stride_size = 10
b_size = 16
acuu- 74


wind_size = 150
stride_size = 10
b_size = 16
aucc - 72


wind_size = 180
stride_size = 10
b_size = 16
aucc - 24


wind_size = 120
stride_size = 20
b_size = 16
aucc - 30


wind_size = 120
stride_size = 5
b_size = 16
aucc - 70

# wind_size = 120, stride_size = 1, b_size = 16, learning rate=1e-4

In [2]:
# ✅ Fully improved model with patient-wise splitting and no shuffling across patients

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split

# 📦 Load dataset

path = r"D:\Data\NYC\KINZ\KINECT_dataset_with_qor15.csv"

#path = "C:/Users/RAZER/Desktop/TEMp/New folder/KINECT_dataset_with_qor15.csv"


dataframe = pd.read_csv(path)
dataframe = dataframe[dataframe['walking_speed'] == 'Fast']
dataframe.dropna(subset=['QoR_class'], inplace=True)

# 📌 Graph structure
joints = ['PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST', 'NECK', 'CLAVICLE_LEFT', 'SHOULDER_LEFT',
          'ELBOW_LEFT', 'WRIST_LEFT', 'HAND_LEFT', 'HANDTIP_LEFT', 'THUMB_LEFT',
          'CLAVICLE_RIGHT', 'SHOULDER_RIGHT', 'ELBOW_RIGHT', 'WRIST_RIGHT', 'HAND_RIGHT',
          'HANDTIP_RIGHT', 'THUMB_RIGHT', 'HIP_LEFT', 'KNEE_LEFT', 'ANKLE_LEFT',
          'FOOT_LEFT', 'HIP_RIGHT', 'KNEE_RIGHT', 'ANKLE_RIGHT', 'FOOT_RIGHT',
          'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT']

edges = [('PELVIS', 'SPINE_NAVAL'), ('SPINE_NAVAL', 'SPINE_CHEST'), ('SPINE_CHEST', 'NECK'),
         ('NECK', 'HEAD'), ('SPINE_CHEST', 'CLAVICLE_LEFT'), ('CLAVICLE_LEFT', 'SHOULDER_LEFT'),
         ('SHOULDER_LEFT', 'ELBOW_LEFT'), ('ELBOW_LEFT', 'WRIST_LEFT'), ('WRIST_LEFT', 'HAND_LEFT'),
         ('HAND_LEFT', 'HANDTIP_LEFT'), ('WRIST_LEFT', 'THUMB_LEFT'), ('SPINE_CHEST', 'CLAVICLE_RIGHT'),
         ('CLAVICLE_RIGHT', 'SHOULDER_RIGHT'), ('SHOULDER_RIGHT', 'ELBOW_RIGHT'), ('ELBOW_RIGHT', 'WRIST_RIGHT'),
         ('WRIST_RIGHT', 'HAND_RIGHT'), ('HAND_RIGHT', 'HANDTIP_RIGHT'), ('WRIST_RIGHT', 'THUMB_RIGHT'),
         ('PELVIS', 'HIP_LEFT'), ('HIP_LEFT', 'KNEE_LEFT'), ('KNEE_LEFT', 'ANKLE_LEFT'),
         ('ANKLE_LEFT', 'FOOT_LEFT'), ('PELVIS', 'HIP_RIGHT'), ('HIP_RIGHT', 'KNEE_RIGHT'),
         ('KNEE_RIGHT', 'ANKLE_RIGHT'), ('ANKLE_RIGHT', 'FOOT_RIGHT'),
         ('HEAD', 'NOSE'), ('HEAD', 'EYE_LEFT'), ('HEAD', 'EYE_RIGHT'), ('HEAD', 'EAR_LEFT'), ('HEAD', 'EAR_RIGHT')]

joint_to_idx = {j: i for i, j in enumerate(joints)}
edge_index = torch.tensor([[joint_to_idx[a], joint_to_idx[b]] for a, b in edges] + 
                          [[joint_to_idx[b], joint_to_idx[a]] for a, b in edges], dtype=torch.long).t()

# 🧱 Dataset
class WindowedSkeletonDataset(torch.utils.data.Dataset):
    def __init__(self, df, window_size=30, stride=15):
        self.samples = []
        self.labels = []
        grouped = df.groupby(['patient_id', 'trial'])
        for _, group in grouped:
            group = group.sort_values('t_uniform')
            if len(group) < window_size:
                continue
            for i in range(0, len(group) - window_size + 1, stride):
                window = group.iloc[i:i+window_size]
                x_seq, aux_seq = [], []
                for _, row in window.iterrows():
                    node_features = [[row[f'{j}_X'], row[f'{j}_Y'], row[f'{j}_Z'], row['t_uniform']] for j in joints]
                    x_seq.append(torch.tensor(node_features, dtype=torch.float))
                    aux_seq.append(torch.tensor([row['left_foot_swing'], row['right_foot_swing'], row['pelvis_pos']], dtype=torch.float))
                label = torch.tensor(window['QoR_class'].mode().values[0], dtype=torch.float)
                self.samples.append((torch.stack(x_seq), torch.stack(aux_seq), label))
                self.labels.append(int(label.item()))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

# 🔀 Patient-wise split
unique_ids = dataframe['patient_id'].unique()
train_ids, test_ids = train_test_split(unique_ids, test_size=0.3, random_state=42)

train_df = dataframe[dataframe['patient_id'].isin(train_ids)]
test_df  = dataframe[dataframe['patient_id'].isin(test_ids)]







#---------------------------------------------------------------------------------------------------------------------------------------------------------

wind_size = 120
stride_size = 10
b_size = 16

#---------------------------------------------------------------------------------------------------------------------------------------------------------






train_dataset = WindowedSkeletonDataset(train_df,window_size=wind_size, stride=stride_size)
test_dataset  = WindowedSkeletonDataset(test_df,window_size=wind_size, stride=stride_size)

# 📦 Dataloaders (no shuffle)
train_loader = DataLoader(train_dataset, batch_size=b_size)
test_loader  = DataLoader(test_dataset, batch_size=b_size)

# 🧠 Model
class WindowedSTGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.gcn = GCNConv(in_channels, hidden_channels)
        self.lstm = nn.LSTM(hidden_channels, hidden_channels, batch_first=True)
        self.aux_fc = nn.Linear(3, hidden_channels)
        self.classifier = nn.Linear(2 * hidden_channels, 1)

    def forward(self, x_seq, aux_seq):
        B, T, N, feat_dim = x_seq.shape
        x_seq = x_seq.view(B * T * N, feat_dim)
        full_edge_index = torch.cat([edge_index + b * N for b in range(B * T)], dim=1).to(x_seq.device)
        x = F.relu(self.gcn(x_seq, full_edge_index))
        x = x.view(B, T, N, -1).mean(2)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        aux = F.relu(self.aux_fc(aux_seq.mean(1)))
        return self.classifier(torch.cat([x, aux], dim=-1)).squeeze()

# ✅ Training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WindowedSTGCN(in_channels=4, hidden_channels=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()

for epoch in range(1, 100):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for x_seq, aux_seq, y in train_loader:
        x_seq, aux_seq, y = x_seq.to(device), aux_seq.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x_seq, aux_seq)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * y.size(0)
        correct += ((torch.sigmoid(out) >= 0.5).float() == y).sum().item()
        total += y.size(0)
    print(f"Epoch {epoch:03d} | Loss: {total_loss / total:.4f} | Accuracy: {correct / total:.4f}")

# 🧪 Final Evaluation
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for x_seq, aux_seq, y in test_loader:
        x_seq, aux_seq = x_seq.to(device), aux_seq.to(device)
        out = torch.sigmoid(model(x_seq, aux_seq))
        pred = (out >= 0.5).float()
        y_true.extend(y.cpu().numpy())
        y_pred.extend(pred.cpu().view(-1).numpy())


print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Good (0)", "Not Good (1)"]))

Epoch 001 | Loss: 31.9955 | Accuracy: 0.5153
Epoch 002 | Loss: 13.5788 | Accuracy: 0.4995
Epoch 003 | Loss: 12.5332 | Accuracy: 0.5000
Epoch 004 | Loss: 11.4555 | Accuracy: 0.4995
Epoch 005 | Loss: 10.4565 | Accuracy: 0.4832
Epoch 006 | Loss: 9.4694 | Accuracy: 0.4761
Epoch 007 | Loss: 8.3562 | Accuracy: 0.4720
Epoch 008 | Loss: 6.1783 | Accuracy: 0.5112
Epoch 009 | Loss: 4.3919 | Accuracy: 0.5183
Epoch 010 | Loss: 3.3671 | Accuracy: 0.5295
Epoch 011 | Loss: 2.9613 | Accuracy: 0.4939
Epoch 012 | Loss: 3.0335 | Accuracy: 0.5056
Epoch 013 | Loss: 2.8485 | Accuracy: 0.5173
Epoch 014 | Loss: 2.8219 | Accuracy: 0.5188
Epoch 015 | Loss: 2.8178 | Accuracy: 0.5178
Epoch 016 | Loss: 2.7999 | Accuracy: 0.5148
Epoch 017 | Loss: 2.7875 | Accuracy: 0.5153
Epoch 018 | Loss: 2.7742 | Accuracy: 0.5158
Epoch 019 | Loss: 2.7624 | Accuracy: 0.5163
Epoch 020 | Loss: 2.7525 | Accuracy: 0.5173
Epoch 021 | Loss: 2.7409 | Accuracy: 0.5183
Epoch 022 | Loss: 2.7348 | Accuracy: 0.5178
Epoch 023 | Loss: 2.7197 | 

In [7]:
from collections import defaultdict
import pandas as pd
import torch

model.eval()
patient_results = defaultdict(list)

# Align test_df index to help lookup patient IDs
test_df = test_df.reset_index(drop=True)

with torch.no_grad():
    for i in range(len(test_dataset)):
        x_seq, aux_seq, y = test_dataset[i]
        patient_id = test_df.iloc[i]['patient_id']
        x_seq, aux_seq = x_seq.unsqueeze(0).to(device), aux_seq.unsqueeze(0).to(device)
        output = model(x_seq, aux_seq)
        prob = torch.sigmoid(output).item()
        pred_label = int(prob >= 0.5)
        true_label = int(y.item())

        patient_results[patient_id].append({
            'pred': pred_label,
            'true': true_label,
            'prob': prob
        })

# Build summary per patient
summary = []
for pid, records in patient_results.items():
    preds = [r['pred'] for r in records]
    trues = [r['true'] for r in records]
    probs = [r['prob'] for r in records]

    correct_count = sum(p == t for p, t in zip(preds, trues))
    total = len(records)
    valid_percentage = 100 * correct_count / total
    invalid_percentage = 100 - valid_percentage

    summary.append({
        'PatientID': pid,
        'Predicted_QoR_class': max(set(preds), key=preds.count),
        'Actual_QoR_class': max(set(trues), key=trues.count),
        'valid_percentage': round(valid_percentage, 2),
        'invalid_percentage': round(invalid_percentage, 2),
        'average_model_output': round(sum(probs) / len(probs), 6)
    })

# Convert to DataFrame and view
report_df = pd.DataFrame(summary).sort_values(by='PatientID').reset_index(drop=True)
display(report_df)

# Save if needed
# report_df.to_csv("QoR_Per_Patient_Report.csv", index=False)


,PatientID,Predicted_QoR_class,Actual_QoR_class,valid_percentage,invalid_percentage,average_model_output
0,001-LO,0,0,68.3,31.7,0.061899


In [8]:
patient_id = test_df.iloc[i]['patient_id']
patient_id

'001-LO'

In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from sklearn.metrics import classification_report
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split

# 📦 Load dataset
path = r"D:\Data\NYC\KINZ\KINECT_dataset_with_qor15.csv"
dataframe = pd.read_csv(path)
dataframe = dataframe[dataframe['walking_speed'] == 'Fast']
dataframe.dropna(subset=['QoR_class'], inplace=True)

# 📌 Graph structure
joints = ['PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST', 'NECK', 'CLAVICLE_LEFT', 'SHOULDER_LEFT',
          'ELBOW_LEFT', 'WRIST_LEFT', 'HAND_LEFT', 'HANDTIP_LEFT', 'THUMB_LEFT',
          'CLAVICLE_RIGHT', 'SHOULDER_RIGHT', 'ELBOW_RIGHT', 'WRIST_RIGHT', 'HAND_RIGHT',
          'HANDTIP_RIGHT', 'THUMB_RIGHT', 'HIP_LEFT', 'KNEE_LEFT', 'ANKLE_LEFT',
          'FOOT_LEFT', 'HIP_RIGHT', 'KNEE_RIGHT', 'ANKLE_RIGHT', 'FOOT_RIGHT',
          'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT']

edges = [('PELVIS', 'SPINE_NAVAL'), ('SPINE_NAVAL', 'SPINE_CHEST'), ('SPINE_CHEST', 'NECK'),
         ('NECK', 'HEAD'), ('SPINE_CHEST', 'CLAVICLE_LEFT'), ('CLAVICLE_LEFT', 'SHOULDER_LEFT'),
         ('SHOULDER_LEFT', 'ELBOW_LEFT'), ('ELBOW_LEFT', 'WRIST_LEFT'), ('WRIST_LEFT', 'HAND_LEFT'),
         ('HAND_LEFT', 'HANDTIP_LEFT'), ('WRIST_LEFT', 'THUMB_LEFT'), ('SPINE_CHEST', 'CLAVICLE_RIGHT'),
         ('CLAVICLE_RIGHT', 'SHOULDER_RIGHT'), ('SHOULDER_RIGHT', 'ELBOW_RIGHT'), ('ELBOW_RIGHT', 'WRIST_RIGHT'),
         ('WRIST_RIGHT', 'HAND_RIGHT'), ('HAND_RIGHT', 'HANDTIP_RIGHT'), ('WRIST_RIGHT', 'THUMB_RIGHT'),
         ('PELVIS', 'HIP_LEFT'), ('HIP_LEFT', 'KNEE_LEFT'), ('KNEE_LEFT', 'ANKLE_LEFT'),
         ('ANKLE_LEFT', 'FOOT_LEFT'), ('PELVIS', 'HIP_RIGHT'), ('HIP_RIGHT', 'KNEE_RIGHT'),
         ('KNEE_RIGHT', 'ANKLE_RIGHT'), ('ANKLE_RIGHT', 'FOOT_RIGHT'),
         ('HEAD', 'NOSE'), ('HEAD', 'EYE_LEFT'), ('HEAD', 'EYE_RIGHT'), ('HEAD', 'EAR_LEFT'), ('HEAD', 'EAR_RIGHT')]

joint_to_idx = {j: i for i, j in enumerate(joints)}
edge_index = torch.tensor([[joint_to_idx[a], joint_to_idx[b]] for a, b in edges] +
                          [[joint_to_idx[b], joint_to_idx[a]] for a, b in edges], dtype=torch.long).t()

# 🧱 Dataset with patient ID tracking
class WindowedSkeletonDataset(torch.utils.data.Dataset):
    def __init__(self, df, window_size=30, stride=15):
        self.samples = []
        grouped = df.groupby(['patient_id', 'trial'])
        for (patient_id, trial), group in grouped:
            group = group.sort_values('t_uniform')
            if len(group) < window_size:
                continue
            for i in range(0, len(group) - window_size + 1, stride):
                window = group.iloc[i:i+window_size]
                x_seq, aux_seq = [], []
                for _, row in window.iterrows():
                    node_features = [[row[f'{j}_X'], row[f'{j}_Y'], row[f'{j}_Z'], row['t_uniform']] for j in joints]
                    x_seq.append(torch.tensor(node_features, dtype=torch.float))
                    aux_seq.append(torch.tensor([row['left_foot_swing'], row['right_foot_swing'], row['pelvis_pos']], dtype=torch.float))
                label = torch.tensor(window['QoR_class'].mode().values[0], dtype=torch.float)
                self.samples.append({
                    'x_seq': torch.stack(x_seq),
                    'aux_seq': torch.stack(aux_seq),
                    'label': label,
                    'patient_id': patient_id
                })

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        return sample['x_seq'], sample['aux_seq'], sample['label'], sample['patient_id']

# 🔀 Patient-wise split
unique_ids = dataframe['patient_id'].unique()
train_ids, test_ids = train_test_split(unique_ids, test_size=0.3, random_state=42)

train_df = dataframe[dataframe['patient_id'].isin(train_ids)]
test_df  = dataframe[dataframe['patient_id'].isin(test_ids)]

# Dataset creation
wind_size = 120
stride_size = 10
b_size = 16

train_dataset = WindowedSkeletonDataset(train_df, window_size=wind_size, stride=stride_size)
test_dataset  = WindowedSkeletonDataset(test_df, window_size=wind_size, stride=stride_size)

train_loader = DataLoader(train_dataset, batch_size=b_size)
test_loader  = DataLoader(test_dataset, batch_size=b_size)

# 🧠 Model
class WindowedSTGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.gcn = GCNConv(in_channels, hidden_channels)
        self.lstm = nn.LSTM(hidden_channels, hidden_channels, batch_first=True)
        self.aux_fc = nn.Linear(3, hidden_channels)
        self.classifier = nn.Linear(2 * hidden_channels, 1)

    def forward(self, x_seq, aux_seq):
        B, T, N, feat_dim = x_seq.shape
        x_seq = x_seq.view(B * T * N, feat_dim)
        full_edge_index = torch.cat([edge_index + b * N for b in range(B * T)], dim=1).to(x_seq.device)
        x = F.relu(self.gcn(x_seq, full_edge_index))
        x = x.view(B, T, N, -1).mean(2)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        aux = F.relu(self.aux_fc(aux_seq.mean(1)))
        return self.classifier(torch.cat([x, aux], dim=-1)).squeeze()

# ✅ Training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WindowedSTGCN(in_channels=4, hidden_channels=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()

for epoch in range(1, 100):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for x_seq, aux_seq, y, _ in train_loader:
        x_seq, aux_seq, y = x_seq.to(device), aux_seq.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x_seq, aux_seq)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * y.size(0)
        correct += ((torch.sigmoid(out) >= 0.5).float() == y).sum().item()
        total += y.size(0)
    print(f"Epoch {epoch:03d} | Loss: {total_loss / total:.4f} | Accuracy: {correct / total:.4f}")

# 🧪 Patient-wise Evaluation
model.eval()
patient_results = defaultdict(list)
with torch.no_grad():
    for i in range(len(test_dataset)):
        x_seq, aux_seq, y, patient_id = test_dataset[i]
        x_seq, aux_seq = x_seq.unsqueeze(0).to(device), aux_seq.unsqueeze(0).to(device)
        output = model(x_seq, aux_seq)
        prob = torch.sigmoid(output).item()
        pred_label = int(prob >= 0.5)
        true_label = int(y.item())

        patient_results[patient_id].append({
            'pred': pred_label,
            'true': true_label,
            'prob': prob
        })

# Build summary
summary = []
for pid, records in patient_results.items():
    preds = [r['pred'] for r in records]
    trues = [r['true'] for r in records]
    probs = [r['prob'] for r in records]

    correct_count = sum(p == t for p, t in zip(preds, trues))
    total = len(records)
    valid_percentage = 100 * correct_count / total
    invalid_percentage = 100 - valid_percentage

    summary.append({
        'PatientID': pid,
        'Predicted_QoR_class': max(set(preds), key=preds.count),
        'Actual_QoR_class': max(set(trues), key=trues.count),
        'valid_percentage': round(valid_percentage, 2),
        'invalid_percentage': round(invalid_percentage, 2),
        'average_model_output': round(sum(probs) / len(probs), 6)
    })

# Convert to DataFrame and display
report_df = pd.DataFrame(summary).sort_values(by='PatientID').reset_index(drop=True)
print(report_df)


Epoch 001 | Loss: 5.2177 | Accuracy: 0.5371
Epoch 002 | Loss: 3.3741 | Accuracy: 0.5498
Epoch 003 | Loss: 3.0358 | Accuracy: 0.5504
Epoch 004 | Loss: 2.8656 | Accuracy: 0.5514
Epoch 005 | Loss: 2.7052 | Accuracy: 0.5509
Epoch 006 | Loss: 2.3874 | Accuracy: 0.5519
Epoch 007 | Loss: 2.2883 | Accuracy: 0.5463
Epoch 008 | Loss: 2.2746 | Accuracy: 0.5422
Epoch 009 | Loss: 2.2652 | Accuracy: 0.5371
Epoch 010 | Loss: 2.2535 | Accuracy: 0.5376
Epoch 011 | Loss: 2.2468 | Accuracy: 0.5387
Epoch 012 | Loss: 2.2417 | Accuracy: 0.5366
Epoch 013 | Loss: 2.2334 | Accuracy: 0.5361
Epoch 014 | Loss: 2.2225 | Accuracy: 0.5341
Epoch 015 | Loss: 2.2103 | Accuracy: 0.5336
Epoch 016 | Loss: 2.1929 | Accuracy: 0.5326
Epoch 017 | Loss: 2.1692 | Accuracy: 0.5270
Epoch 018 | Loss: 2.1003 | Accuracy: 0.5153
Epoch 019 | Loss: 2.1372 | Accuracy: 0.5051
Epoch 020 | Loss: 2.1285 | Accuracy: 0.5025
Epoch 021 | Loss: 2.1198 | Accuracy: 0.4995
Epoch 022 | Loss: 2.1169 | Accuracy: 0.4980
Epoch 023 | Loss: 2.1142 | Accur

In [12]:
# ✅ Fully improved model with patient-wise splitting and no shuffling across patients

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split

# 📦 Load dataset

path = r"D:\Data\NYC\KINZ\KINECT_dataset_with_qor15.csv"

dataframe = pd.read_csv(path)
dataframe = dataframe[dataframe['walking_speed'] == 'Fast']
dataframe.dropna(subset=['QoR_class'], inplace=True)

# 📌 Graph structure
joints = ['PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST', 'NECK', 'CLAVICLE_LEFT', 'SHOULDER_LEFT',
          'ELBOW_LEFT', 'WRIST_LEFT', 'HAND_LEFT', 'HANDTIP_LEFT', 'THUMB_LEFT',
          'CLAVICLE_RIGHT', 'SHOULDER_RIGHT', 'ELBOW_RIGHT', 'WRIST_RIGHT', 'HAND_RIGHT',
          'HANDTIP_RIGHT', 'THUMB_RIGHT', 'HIP_LEFT', 'KNEE_LEFT', 'ANKLE_LEFT',
          'FOOT_LEFT', 'HIP_RIGHT', 'KNEE_RIGHT', 'ANKLE_RIGHT', 'FOOT_RIGHT',
          'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT']

edges = [('PELVIS', 'SPINE_NAVAL'), ('SPINE_NAVAL', 'SPINE_CHEST'), ('SPINE_CHEST', 'NECK'),
         ('NECK', 'HEAD'), ('SPINE_CHEST', 'CLAVICLE_LEFT'), ('CLAVICLE_LEFT', 'SHOULDER_LEFT'),
         ('SHOULDER_LEFT', 'ELBOW_LEFT'), ('ELBOW_LEFT', 'WRIST_LEFT'), ('WRIST_LEFT', 'HAND_LEFT'),
         ('HAND_LEFT', 'HANDTIP_LEFT'), ('WRIST_LEFT', 'THUMB_LEFT'), ('SPINE_CHEST', 'CLAVICLE_RIGHT'),
         ('CLAVICLE_RIGHT', 'SHOULDER_RIGHT'), ('SHOULDER_RIGHT', 'ELBOW_RIGHT'), ('ELBOW_RIGHT', 'WRIST_RIGHT'),
         ('WRIST_RIGHT', 'HAND_RIGHT'), ('HAND_RIGHT', 'HANDTIP_RIGHT'), ('WRIST_RIGHT', 'THUMB_RIGHT'),
         ('PELVIS', 'HIP_LEFT'), ('HIP_LEFT', 'KNEE_LEFT'), ('KNEE_LEFT', 'ANKLE_LEFT'),
         ('ANKLE_LEFT', 'FOOT_LEFT'), ('PELVIS', 'HIP_RIGHT'), ('HIP_RIGHT', 'KNEE_RIGHT'),
         ('KNEE_RIGHT', 'ANKLE_RIGHT'), ('ANKLE_RIGHT', 'FOOT_RIGHT'),
         ('HEAD', 'NOSE'), ('HEAD', 'EYE_LEFT'), ('HEAD', 'EYE_RIGHT'), ('HEAD', 'EAR_LEFT'), ('HEAD', 'EAR_RIGHT')]

joint_to_idx = {j: i for i, j in enumerate(joints)}
edge_index = torch.tensor([[joint_to_idx[a], joint_to_idx[b]] for a, b in edges] + 
                          [[joint_to_idx[b], joint_to_idx[a]] for a, b in edges], dtype=torch.long).t()

# 🧱 Dataset
class WindowedSkeletonDataset(torch.utils.data.Dataset):
    def __init__(self, df, window_size=30, stride=15):
        self.samples = []
        self.labels = []
        grouped = df.groupby(['patient_id', 'trial'])
        for _, group in grouped:
            group = group.sort_values('t_uniform')
            if len(group) < window_size:
                continue
            for i in range(0, len(group) - window_size + 1, stride):
                window = group.iloc[i:i+window_size]
                x_seq, aux_seq = [], []
                for _, row in window.iterrows():
                    node_features = [[row[f'{j}_X'], row[f'{j}_Y'], row[f'{j}_Z'], row['t_uniform']] for j in joints]
                    x_seq.append(torch.tensor(node_features, dtype=torch.float))
                    aux_seq.append(torch.tensor([row['left_foot_swing'], row['right_foot_swing'], row['pelvis_pos']], dtype=torch.float))
                label = torch.tensor(window['QoR_class'].mode().values[0], dtype=torch.float)
                self.samples.append((torch.stack(x_seq), torch.stack(aux_seq), label))
                self.labels.append(int(label.item()))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

# 🔀 Patient-wise split
unique_ids = dataframe['patient_id'].unique()
train_ids, test_ids = train_test_split(unique_ids, test_size=0.3, random_state=42)

train_df = dataframe[dataframe['patient_id'].isin(train_ids)]
test_df  = dataframe[dataframe['patient_id'].isin(test_ids)]

wind_size = 120
stride_size = 10
b_size = 16

train_dataset = WindowedSkeletonDataset(train_df, window_size=wind_size, stride=stride_size)
test_dataset  = WindowedSkeletonDataset(test_df, window_size=wind_size, stride=stride_size)

train_loader = DataLoader(train_dataset, batch_size=b_size)
test_loader  = DataLoader(test_dataset, batch_size=b_size)

# 🧠 Model
class WindowedSTGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.gcn = GCNConv(in_channels, hidden_channels)
        self.lstm = nn.LSTM(hidden_channels, hidden_channels, batch_first=True)
        self.aux_fc = nn.Linear(3, hidden_channels)
        self.classifier = nn.Linear(2 * hidden_channels, 1)

    def forward(self, x_seq, aux_seq):
        B, T, N, feat_dim = x_seq.shape
        x_seq = x_seq.view(B * T * N, feat_dim)
        full_edge_index = torch.cat([edge_index + b * N for b in range(B * T)], dim=1).to(x_seq.device)
        x = F.relu(self.gcn(x_seq, full_edge_index))
        x = x.view(B, T, N, -1).mean(2)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        aux = F.relu(self.aux_fc(aux_seq.mean(1)))
        return self.classifier(torch.cat([x, aux], dim=-1)).squeeze()

# ✅ Training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WindowedSTGCN(in_channels=4, hidden_channels=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()

for epoch in range(1, 100):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for x_seq, aux_seq, y in train_loader:
        x_seq, aux_seq, y = x_seq.to(device), aux_seq.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x_seq, aux_seq)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * y.size(0)
        correct += ((torch.sigmoid(out) >= 0.5).float() == y).sum().item()
        total += y.size(0)
    print(f"Epoch {epoch:03d} | Loss: {total_loss / total:.4f} | Accuracy: {correct / total:.4f}")

# 🧪 Final Evaluation
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for x_seq, aux_seq, y in test_loader:
        x_seq, aux_seq = x_seq.to(device), aux_seq.to(device)
        out = torch.sigmoid(model(x_seq, aux_seq))
        pred = (out >= 0.5).float()
        y_true.extend(y.cpu().numpy())
        y_pred.extend(pred.cpu().view(-1).numpy())

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Good (0)", "Not Good (1)"]))


Epoch 001 | Loss: 62.4791 | Accuracy: 0.4730
Epoch 002 | Loss: 3.4382 | Accuracy: 0.4817
Epoch 003 | Loss: 3.4234 | Accuracy: 0.4807
Epoch 004 | Loss: 3.3009 | Accuracy: 0.4852
Epoch 005 | Loss: 3.1941 | Accuracy: 0.4832
Epoch 006 | Loss: 3.0916 | Accuracy: 0.4852
Epoch 007 | Loss: 3.2550 | Accuracy: 0.4766
Epoch 008 | Loss: 3.0195 | Accuracy: 0.4975
Epoch 009 | Loss: 3.2109 | Accuracy: 0.4847
Epoch 010 | Loss: 2.9088 | Accuracy: 0.4964
Epoch 011 | Loss: 3.1530 | Accuracy: 0.5005
Epoch 012 | Loss: 2.8007 | Accuracy: 0.4852
Epoch 013 | Loss: 3.1228 | Accuracy: 0.5071
Epoch 014 | Loss: 2.7233 | Accuracy: 0.4822
Epoch 015 | Loss: 3.1006 | Accuracy: 0.5178
Epoch 016 | Loss: 2.6839 | Accuracy: 0.4771
Epoch 017 | Loss: 3.0782 | Accuracy: 0.5209
Epoch 018 | Loss: 2.6678 | Accuracy: 0.4746
Epoch 019 | Loss: 3.0577 | Accuracy: 0.5219
Epoch 020 | Loss: 2.6641 | Accuracy: 0.4791
Epoch 021 | Loss: 3.0399 | Accuracy: 0.5244
Epoch 022 | Loss: 2.6644 | Accuracy: 0.4817
Epoch 023 | Loss: 3.0277 | Accu

In [18]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from sklearn.metrics import classification_report
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split

# Load dataset
path = r"D:\Data\NYC\KINZ\KINECT_dataset_with_qor15.csv"
dataframe = pd.read_csv(path)
dataframe = dataframe[dataframe['walking_speed'] == 'Fast']
dataframe.dropna(subset=['QoR_class'], inplace=True)

# Graph structure
joints = ['PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST', 'NECK', 'CLAVICLE_LEFT', 'SHOULDER_LEFT',
          'ELBOW_LEFT', 'WRIST_LEFT', 'HAND_LEFT', 'HANDTIP_LEFT', 'THUMB_LEFT',
          'CLAVICLE_RIGHT', 'SHOULDER_RIGHT', 'ELBOW_RIGHT', 'WRIST_RIGHT', 'HAND_RIGHT',
          'HANDTIP_RIGHT', 'THUMB_RIGHT', 'HIP_LEFT', 'KNEE_LEFT', 'ANKLE_LEFT',
          'FOOT_LEFT', 'HIP_RIGHT', 'KNEE_RIGHT', 'ANKLE_RIGHT', 'FOOT_RIGHT',
          'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT']

edges = [('PELVIS', 'SPINE_NAVAL'), ('SPINE_NAVAL', 'SPINE_CHEST'), ('SPINE_CHEST', 'NECK'),
         ('NECK', 'HEAD'), ('SPINE_CHEST', 'CLAVICLE_LEFT'), ('CLAVICLE_LEFT', 'SHOULDER_LEFT'),
         ('SHOULDER_LEFT', 'ELBOW_LEFT'), ('ELBOW_LEFT', 'WRIST_LEFT'), ('WRIST_LEFT', 'HAND_LEFT'),
         ('HAND_LEFT', 'HANDTIP_LEFT'), ('WRIST_LEFT', 'THUMB_LEFT'), ('SPINE_CHEST', 'CLAVICLE_RIGHT'),
         ('CLAVICLE_RIGHT', 'SHOULDER_RIGHT'), ('SHOULDER_RIGHT', 'ELBOW_RIGHT'), ('ELBOW_RIGHT', 'WRIST_RIGHT'),
         ('WRIST_RIGHT', 'HAND_RIGHT'), ('HAND_RIGHT', 'HANDTIP_RIGHT'), ('WRIST_RIGHT', 'THUMB_RIGHT'),
         ('PELVIS', 'HIP_LEFT'), ('HIP_LEFT', 'KNEE_LEFT'), ('KNEE_LEFT', 'ANKLE_LEFT'),
         ('ANKLE_LEFT', 'FOOT_LEFT'), ('PELVIS', 'HIP_RIGHT'), ('HIP_RIGHT', 'KNEE_RIGHT'),
         ('KNEE_RIGHT', 'ANKLE_RIGHT'), ('ANKLE_RIGHT', 'FOOT_RIGHT'),
         ('HEAD', 'NOSE'), ('HEAD', 'EYE_LEFT'), ('HEAD', 'EYE_RIGHT'), ('HEAD', 'EAR_LEFT'), ('HEAD', 'EAR_RIGHT')]

joint_to_idx = {j: i for i, j in enumerate(joints)}
edge_index = torch.tensor([[joint_to_idx[a], joint_to_idx[b]] for a, b in edges] + 
                          [[joint_to_idx[b], joint_to_idx[a]] for a, b in edges], dtype=torch.long).t()

# Dataset
class WindowedSkeletonDataset(torch.utils.data.Dataset):
    def __init__(self, df, window_size=120, stride=10):
        self.samples = []
        grouped = df.groupby(['patient_id', 'trial'])
        for (pid, trial), group in grouped:
            group = group.sort_values('t_uniform')
            if len(group) < window_size:
                continue
            for i in range(0, len(group) - window_size + 1, stride):
                window = group.iloc[i:i+window_size]
                x_seq, aux_seq = [], []
                for _, row in window.iterrows():
                    node_features = [[row[f'{j}_X'], row[f'{j}_Y'], row[f'{j}_Z'], row['t_uniform']] for j in joints]
                    x_seq.append(torch.tensor(node_features, dtype=torch.float))
                    aux_seq.append(torch.tensor([row['left_foot_swing'], row['right_foot_swing'], row['pelvis_pos']], dtype=torch.float))
                label = torch.tensor(window['QoR_class'].mode().values[0], dtype=torch.float)
                self.samples.append((torch.stack(x_seq), torch.stack(aux_seq), label, pid))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

# Patient-wise split
unique_ids = dataframe['patient_id'].unique()
train_ids, test_ids = train_test_split(unique_ids, test_size=0.3, random_state=42)

train_df = dataframe[dataframe['patient_id'].isin(train_ids)]
test_df  = dataframe[dataframe['patient_id'].isin(test_ids)]

train_dataset = WindowedSkeletonDataset(train_df)
test_dataset  = WindowedSkeletonDataset(test_df)

# Model
class WindowedSTGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.gcn = GCNConv(in_channels, hidden_channels)
        self.lstm = nn.LSTM(hidden_channels, hidden_channels, batch_first=True)
        self.aux_fc = nn.Linear(3, hidden_channels)
        self.classifier = nn.Linear(2 * hidden_channels, 1)

    def forward(self, x_seq, aux_seq):
        B, T, N, feat_dim = x_seq.shape
        x_seq = x_seq.view(B * T * N, feat_dim)
        full_edge_index = torch.cat([edge_index + b * N for b in range(B * T)], dim=1).to(x_seq.device)
        x = F.relu(self.gcn(x_seq, full_edge_index))
        x = x.view(B, T, N, -1).mean(2)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        aux = F.relu(self.aux_fc(aux_seq.mean(1)))
        return self.classifier(torch.cat([x, aux], dim=-1)).view(-1)  # <-- ensure shape [B]

# Training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WindowedSTGCN(in_channels=4, hidden_channels=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()

for epoch in range(1, 101):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for x_seq, aux_seq, y, _ in train_dataset:
        x_seq, aux_seq, y = x_seq.unsqueeze(0).to(device), aux_seq.unsqueeze(0).to(device), y.unsqueeze(0).to(device)
        optimizer.zero_grad()
        out = model(x_seq, aux_seq)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct += ((torch.sigmoid(out) >= 0.5).float() == y).sum().item()
        total += 1
    print(f"Epoch {epoch:03d} | Loss: {total_loss / total:.4f} | Accuracy: {correct / total:.4f}")

# Evaluation - Patient-wise report
model.eval()
patient_results = defaultdict(list)

with torch.no_grad():
    for x_seq, aux_seq, y, pid in test_dataset:
        x_seq, aux_seq = x_seq.unsqueeze(0).to(device), aux_seq.unsqueeze(0).to(device)
        output = model(x_seq, aux_seq)
        prob = torch.sigmoid(output).item()
        pred_label = int(prob >= 0.5)
        true_label = int(y.item())

        patient_results[pid].append({
            'pred': pred_label,
            'true': true_label,
            'prob': prob
        })

# Summarize per patient
summary = []
for pid, records in patient_results.items():
    preds = [r['pred'] for r in records]
    trues = [r['true'] for r in records]
    probs = [r['prob'] for r in records]

    correct_count = sum(p == t for p, t in zip(preds, trues))
    total = len(records)
    valid_percentage = 100 * correct_count / total
    invalid_percentage = 100 - valid_percentage

    summary.append({
        'PatientID': pid,
        'Predicted_QoR_class': max(set(preds), key=preds.count),
        'Actual_QoR_class': max(set(trues), key=trues.count),
        'valid_percentage': round(valid_percentage, 2),
        'invalid_percentage': round(invalid_percentage, 2),
        'average_model_output': round(sum(probs) / len(probs), 6)
    })

# Display
report_df = pd.DataFrame(summary).sort_values(by='PatientID').reset_index(drop=True)
print(report_df)



Epoch 001 | Loss: 14.7414 | Accuracy: 0.7248
Epoch 002 | Loss: 14.6504 | Accuracy: 0.7309
Epoch 003 | Loss: 13.6332 | Accuracy: 0.7309
Epoch 004 | Loss: 13.6377 | Accuracy: 0.7284
Epoch 005 | Loss: 12.9257 | Accuracy: 0.7304
Epoch 006 | Loss: 12.5999 | Accuracy: 0.7325
Epoch 007 | Loss: 12.3075 | Accuracy: 0.7335
Epoch 008 | Loss: 12.2647 | Accuracy: 0.7299
Epoch 009 | Loss: 11.9928 | Accuracy: 0.7304
Epoch 010 | Loss: 11.7584 | Accuracy: 0.7304
Epoch 011 | Loss: 11.5194 | Accuracy: 0.7309
Epoch 012 | Loss: 11.5057 | Accuracy: 0.7330
Epoch 013 | Loss: 10.9004 | Accuracy: 0.7370
Epoch 014 | Loss: 10.6310 | Accuracy: 0.7391
Epoch 015 | Loss: 10.5328 | Accuracy: 0.7380
Epoch 016 | Loss: 10.4476 | Accuracy: 0.7375
Epoch 017 | Loss: 10.1943 | Accuracy: 0.7370
Epoch 018 | Loss: 9.9294 | Accuracy: 0.7370
Epoch 019 | Loss: 9.8819 | Accuracy: 0.7350
Epoch 020 | Loss: 9.6589 | Accuracy: 0.7350
Epoch 021 | Loss: 9.4688 | Accuracy: 0.7350
Epoch 022 | Loss: 9.3423 | Accuracy: 0.7340
Epoch 023 | Los